In [14]:
# Importing Libraries
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import cv2
import os
import PIL
import random
from time import time
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [15]:
# Determining the data paths
train_path = "../input/sprints-ai-and-ml-competition-2022/Scenes training set/Scenes training set"
test_path = "../input/sprints-ai-and-ml-competition-2022/Scenes testing test/Scenes testing test"

In [16]:
img_width = 150 # Define the width of images 
img_height = 150 # Define the height of images 
batch_size = 32 # Define the size of the batch

In [17]:
# Loading data images and labels function for training
def load_img(pth):
    Img = []
    Lbl = []  
    lbl = 0       
    for i in os.listdir(pth): 
            print(i)
            if i == 'buildings': 
                lbl = 0
            elif i == 'forest':
                lbl = 1
            elif i == 'glacier':
                lbl = 2
            elif i == 'mountain':
                lbl = 3
            elif i == 'sea':
                lbl = 4
            elif i == 'street':
                lbl = 5
            
            for m in os.listdir(pth+'/'+i): 
               
                image = cv2.imread(pth+'/'+i+'/'+m) 
                image = cv2.resize(image,(150,150)) 
                Img.append(image)
                Lbl.append(lbl)
        
    return shuffle(Img, Lbl, random_state = 1) 

def class_lbl(index):
    labels = {0:'buildings', 1:'forest', 2:'glacier', 3:'mountain', 4:'sea',5:'street' }
    
    return labels[index]

In [18]:
# Converting images and labels into NumPy arrays
img, lbl = load_img(train_path) 
img = np.array(img) 
lbl = np.array(lbl)

In [19]:
# Plotting random images and their labels
fig ,ax = plt.subplots(3,3)
fig.subplots_adjust(0,0,3,3)
for n in range (3):
    for i in range(3):
        var = random.randint(0,len(img))
        ax[n,i].imshow(img[var])
        ax[n,i].set_title(class_lbl(lbl[var]))
    

In [20]:
# Building VGG19 model
from tensorflow.keras.applications import VGG19

My_model=VGG19(input_shape = (150, 150, 3), 
                        include_top = False, 
                        weights = 'imagenet')

for layer in My_model.layers:
     layer.trainable = False

My_model.summary()

In [21]:
# Adding some layer (Tuning the model)
from tensorflow.keras import Model
last_layer = My_model.get_layer('block5_pool')
last_output= last_layer.output
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(6, activation='softmax')(x)           
vgg = Model(My_model.input, x)

In [ ]:
# Compiling the model
vgg.compile(optimizer = RMSprop(learning_rate=0.0001),loss = 'sparse_categorical_crossentropy',metrics = ['acc'])
vgg.summary()

In [22]:
# Fitting the model
vgg.fit(img,lbl,epochs =15,validation_split=0.2)

In [23]:
# Reading the empty submit file
df = pd.read_csv('../input/sprints-ai-and-ml-competition-2022/submit.csv')

In [24]:
# Predicting the test set and storing the predictions in a DataFrame
from tensorflow import expand_dims
images=os.listdir(test_path)

for i,image in enumerate(images):
    arr=cv2.imread(os.path.join(test_path,image))
    arr = cv2.resize(arr, (150, 150))
    arr = expand_dims(arr, axis=0)
    name=image
    shape=arr.shape
    prediction=np.argmax(vgg.predict(arr))
    df.loc[i,"Image"]=str(name)
    df.loc[i,"Label"]=prediction

In [25]:
# Appending results to a CSV file
df.to_csv('./submit.csv',index = False)